In [2]:
import numpy as np
import pandas as pd
import datetime
import random
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
plt.style.use('ggplot')
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
import shap
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore", category=FutureWarning)

In [17]:
df = pd.read_csv(r"./GIT_data/data_set.csv")

print("Number of df samples are",df.shape)

categorical_features = ['State_Factor', 'building_class', 'facility_type']
numerical_features=df.select_dtypes('number').columns

Number of df samples are (75757, 64)


## Data preprocess wrapper

In [18]:
# Define angle to direction
def angle_to_direction(angle):
    # Ensure the angle is within the range [0, 360)
    normalized_angle = angle % 360

    # Define directional ranges
    directions = {
        (337.5, 22.5): 'north',
        (22.5, 67.5): 'northeast',
        (67.5, 112.5): 'east',
        (112.5, 157.5): 'southeast',
        (157.5, 202.5): 'south',
        (202.5, 247.5): 'southwest',
        (247.5, 292.5): 'west',
        (292.5, 337.5): 'northwest'
    }

    # Determine the direction based on the angle
    for angle_range, direction_name in directions.items():
        if normalized_angle >= angle_range[0] and normalized_angle < angle_range[1]:
            return direction_name

    # If the angle is not within any defined range, return None or handle it accordingly
    return 'unknown'

# Change wind speed angle to directions
df['direction_max_wind_speed_cate'] = df['direction_max_wind_speed'].apply(angle_to_direction)
df['direction_peak_wind_speed_cate'] = df['direction_peak_wind_speed'].apply(angle_to_direction)

# New categorical features
categorical_features_new = categorical_features + ['direction_max_wind_speed_cate', 'direction_peak_wind_speed_cate']
categorical_features_new

# Onehot Encoder to categorical features
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, categories='auto')
one_hot_facility_type = encoder.fit_transform(df[categorical_features_new])
for i, category in enumerate(encoder.categories_[0]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]
for i, category in enumerate(encoder.categories_[1]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]
for i, category in enumerate(encoder.categories_[2]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]
for i, category in enumerate(encoder.categories_[3]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]
for i, category in enumerate(encoder.categories_[4]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]


# Missing value 
missing_columns = [col for col in df.columns if df[col].isnull().any()]
missingvalues_count =df.isna().sum()
missingValues_df = pd.DataFrame(missingvalues_count.rename('Null Values Count')).loc[missingvalues_count.ne(0)]
missingValues_df.style.background_gradient(cmap="Pastel1")

df['year_built'] =df['year_built'].replace(np.nan, 2022)
null_col=['energy_star_rating','direction_max_wind_speed','direction_peak_wind_speed','max_wind_speed','days_with_fog']
imputer = SimpleImputer()
imputer.fit(df[null_col])
data_transformed = imputer.transform(df[null_col])
df[null_col] = pd.DataFrame(data_transformed)

# Standardize numberical values
import copy
y = df["site_eui"]
train = df.drop(["site_eui","id", "Year_Factor", "direction_max_wind_speed", "direction_peak_wind_speed", "State_Factor", "building_class", "facility_type", 
                 "direction_peak_wind_speed_cate", "direction_max_wind_speed_cate"],axis =1)
trainnames = copy.deepcopy(train)
scaler = StandardScaler()
train = scaler.fit_transform(train)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size = 0.2, random_state = 50)


In [21]:
X_test_ft=pd.DataFrame(X_test,columns=trainnames.columns)
X_test_ft.shape

(15152, 128)

In [22]:
import xgboost
xgboost_model = xgboost.XGBRegressor(n_estimators=200, learning_rate=0.02, gamma=0, subsample=0.75,
                           colsample_bytree=0.4, max_depth=5)
xgboost_model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.4, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.02, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [23]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 250],
    'learning_rate': [0.01, 0.05],
    'max_depth': [3, 4],
    'subsample': [0.6, 0.9],
    'colsample_bytree': [0.2, 0.5],
    'gamma': [0.01, 0.05]
}

# Create an XGBoost regressor
xgboost_model = xgboost.XGBRegressor()

# Create GridSearchCV
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
best_model = grid_search.best_estimator_


# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE) on Test Set:", rmse)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_spar

Best Hyperparameters: {'colsample_bytree': 0.5, 'gamma': 0.01, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9}
Root Mean Squared Error (RMSE) on Test Set: 42.94350134575407
